## Computational Chemistry for Experimentalists
## Chapter 5: Geometry Optimization

Determining a stable (local minimum) molecular geometry is an essential first step of most molecular simulations. This worked example shows a few representative optimized geometries and local minima . 

This first block imports all of the necessary Python modules. If these aren't installed, this will fail. 

In [ ]:
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem import AllChem
from pyscf import gto,scf
from pyscf.tools import cubegen 
import py3Dmol
import numpy 
import matplotlib.pyplot as plt
from pyscf.geomopt.geometric_solver import optimize
from rdkit.Geometry import Point3D

### Part 1: Optimization on a one-dimensional PES, carbon monoxide 

Predict the ground-state potential energy surface for carbon monoxide, by computing the energy at multiple bond lengths. We'll compute the energy of CO molecule relative to the energies of isolated ground-state C and O atoms. 

In [ ]:
basis='3-21g'
hartreetokcalmol = 627.5095
mc=gto.Mole(atom='C',spin=2,basis=basis)
mc.build()
mfc=scf.UHF(mc)
mfc.kernel()
mo=gto.Mole(atom='O',spin=2,basis=basis)
mo.build()
mfo=scf.UHF(mo)
mfo.kernel()
Eat=mfc.e_tot+mfo.e_tot


In [ ]:
rs=[0.5,0.6,0.7,0.8,0.9,1.0,1.05,1.1,1.15,1.2,1.3,1.4,1.5,1.6,1.7,1.8,2.0,2.2,2.3,2.5,2.7,3.0]
des=[]
Pg=None
for r in rs:
    geom='C 0.0 0.0 0.0; O 0.0 0.0 %.2f'%(r)
    mco=gto.Mole(atom=geom,basis=basis)
    mco.build()
    nao=mco.nao
    mfco=scf.RHF(mco)
    mfco.kernel(dm0=Pg)
    Pg=mfco.make_rdm1() # Use the current density matrix as the next guess 
    de=hartreetokcalmol*(mfco.e_tot-Eat)
    des.append(de)

Plot the potential energy surface obtained. Computing all of these points for a high-dimensional PES would be very expensive, which is why we use "geometry optimization" algorithms instead. 

The dissociation limit would be zero if we used a more accurate electronic structure approximation. We'll discuss this much more in a later module 

In [ ]:
plt.plot(rs,des)
plt.xlabel('C=O bond length (Angstrom)')
plt.ylabel('Bond energy (kcal/mol)')
plt.axhline(y=0, color='black',linestyle='-')
ax=plt.gca()
ax.set_ylim(-200,300)
plt.show()

Do a PySCF geometry optimization starting from a C=O bond length 1.7 Angstrom. Use the Jupyter 'magic command' to collect geometry and energy from each step of the optimization into a file 

In [ ]:
%%capture cap 
mco=gto.Mole(atom='C 0.0 0.0 0.0; O 0.0 0.0 1.7',basis=basis)
mco.build()
mfco=scf.RHF(mco)
#mfco.kernel()
optimize(mfco)

In [ ]:
#cap()
with open('COopt.txt', 'w') as file:
    file.write(cap.stdout)

Read the file to determine the dissociaton energy and C=O bond length at each step of the geometry optimization 

In [ ]:
DEopt=[]
Csopt=[]
Osopt=[]
with open('COopt.txt', 'r') as file:
    for line in file:
        fields=line.split()
        if(len(fields)>0):
            if(fields[0]=='cycle'):
                DE=hartreetokcalmol*(float(fields[4])-Eat)
                DEopt.append(DE)
            if(fields[0]=='C'):
                Csopt.append(numpy.array((float(fields[2]),float(fields[3]),float(fields[4]))))
            if(fields[0]=='O'):
                Osopt.append(numpy.array((float(fields[2]),float(fields[3]),float(fields[4]))))
Rsopt=[]
for i in range(len(Csopt)):
    v=Csopt[i]-Osopt[i]
    R=numpy.dot(v,v)**0.5
    Rsopt.append(R)
                             
#print(Rsopt
labs=numpy.array(range(len(Csopt)))+1

Plot each step as a point on the potential energy surface, labeled with step number. The algorithm "walks downhill" in steps 1-3, overshoots the minimum at step 4, then moves back at step 5 and converges by step 7. 

In [ ]:
fig=plt.figure()
ax1=fig.add_subplot(111)
ax1.plot(rs,des,label="Computed PES")
ax1.scatter(Rsopt,DEopt,marker='o',label="Optimization points")
ax1.set_ylim(-200,300)
for i, txt in enumerate(labs):
    ax1.annotate(txt, (Rsopt[i], DEopt[i]),fontsize=14)
plt.xlabel('C=O bond length (Angstrom)')
plt.ylabel('Bond energy (kcal/mol)')
plt.axhline(y=0, color='black',linestyle='-')
plt.legend(loc='upper right')
plt.show()

### Part 2: Optimizing multiple local minima: thioformic acid and 1-pentane

In this example, we consider  molecules with multiple local minima. While these may or may not interconvert in the gas phase or solution at normal temperatures, they always need to be treated separately in geometry optimizations.

For thioformic acid, we generate two closely related initial geometries, and optimize them to two different isomers. This first block generates the default geometry for thioformic acid, and a modified geometry with a stretched H-O bond 

In [ ]:
m=Chem.MolFromSmiles('C(=S)O')
ma=Chem.AddHs(m)
AllChem.EmbedMolecule(ma)
mb=Chem.AddHs(m)
AllChem.EmbedMolecule(mb)
mb.GetConformer().SetAtomPosition(4,Point3D(-1.1,0.3,1.3))
mbla=Chem.MolToMolBlock(ma)
mblb=Chem.MolToMolBlock(mb)
p=py3Dmol.view(width=400,height=400,viewergrid=(1,2))
p.addModel(mbla,'sdf',viewer=(0,0))
p.addModel(mblb,'sdf',viewer=(0,1))
p.setStyle({'stick':{},'sphere':{"scale":0.3}})
p.zoomTo()
p.show()

Convert each RDKit geometry into a PySCF geometry and optimize 

In [ ]:
elements = [atom.GetSymbol() for atom in ma.GetAtoms()]
ca = ma.GetConformer().GetPositions()
cb = mb.GetConformer().GetPositions()
#print(coordinates)
aa = [(element, coordinate) for element, coordinate in zip(elements, ca)]
ab = [(element, coordinate) for element, coordinate in zip(elements, cb)]
print(atoms)
m3a = gto.Mole(basis="STO-3G")
m3a.atom = aa
m3a.build();
m3b = gto.Mole(basis="STO-3G")
m3b.atom = ab
m3b.build();
mf3a=scf.RHF(m3a)
mf3b=scf.RHF(m3b)
m4a=optimize(mf3a)
m4b=optimize(mf3b)


View each optimized geometry. While the code  draws a little stick between H and O, not H and S, the two optimized geometries are consistent with the tautomers. This is OK, little sticks are not bonds! 

In [ ]:
ca = ma.GetConformer()
cb = mb.GetConformer()
nca=m4a.atom_coords() *0.529177
ncb=m4b.atom_coords() *0.529177
for i in range(ma.GetNumAtoms()):
    ca.SetAtomPosition(i,Point3D(nca[i,0],nca[i,1],nca[i,2]))
    cb.SetAtomPosition(i,Point3D(ncb[i,0],ncb[i,1],ncb[i,2]))
mbla=Chem.MolToMolBlock(ma)
mblb=Chem.MolToMolBlock(mb)
p=py3Dmol.view(width=400,height=400,viewergrid=(1,2))
p.addModel(mbla,'sdf',viewer=(0,0))
p.addModel(mblb,'sdf',viewer=(0,1))
p.setStyle({'stick':{},'sphere':{"scale":0.3}})
p.zoomTo()
p.show()

For pentane, we  generate 100 conformers with the RDKit, optimize each with a MM force field, and view the first few nondegenerate optimized conformers.

In [ ]:
m=Chem.MolFromSmiles('CCCCC')
m2=Chem.AddHs(m)
confids=AllChem.EmbedMultipleConfs(m2,numConfs=100)
print('Number of conformers: %d'%(len(confids)))
uniqueEs=[]
uniqueIDs=[]
for confid in confids:
    AllChem.MMFFOptimizeMolecule(m2,confId=confid)
    ff = AllChem.MMFFGetMoleculeForceField(m2, AllChem.MMFFGetMoleculeProperties(m2), confId=confid)
    E=ff.CalcEnergy()
    keep=1
    for Eold in uniqueEs:
        if((E-Eold)**2<0.000001):
            keep=0
    if(keep>0):
        uniqueEs.append(E)
        uniqueIDs.append(confid)
        
# Sort by energy 
sortedEs=[(x,y) for x,y in sorted(zip(uniqueEs,uniqueIDs))]
Emin=sortedEs[0][0]
print('Lowest energy: %.4f'%(Emin))
    
p = py3Dmol.view(width=600,height=200,viewergrid=(1,4))
for ij in range(4):
    i=ij%4
    j=ij/4
    ss=sortedEs[ij]
    confid=ss[1]
    E=ss[0]
    DE=(E-Emin)
    DElabel='%.2f'%(DE)
    p.addModel(Chem.MolToMolBlock(m2,confId=confid), 'sdf',viewer=(j,i))
    p.addLabel(DElabel,{'inFront':True,'fontColor':'black','backgroundColor':'white'},viewer=(j,i))
    p.setStyle({'stick':{},'sphere':{"scale":0.3}},viewer=(j,i))
p.zoomTo()
#p.update()
p.render()


## Part 3: A more complicated optimization, saqinavir 

Optimizing the geometries of large molecules can be challenging. In this example, I show the RDKit and PySCF optimization of a fragment of the anti-AIDS drug saquinavir. I intentionally include a mistake, leaving out two hydrogen atoms. The molecule is still a singlet with charge +1, however, the geometry is severely distorted due to the diradical character. 

I start the geometry optimization from the canonical SMILES available from PubChem 

In [ ]:
#m=Chem.MolFromSmiles('CC(C)(C)NC(=O)C1CC2C[CH]CCC2CN1CC(C(CC3=CC=CC=C3)[N]C(=O)C(CC(=O)[NH3+])NC(=O)C4=NC5=CC=CC=C5C=C4)O')
m=Chem.MolFromSmiles('CC(C)(C)NC(=O)C1CC2C[CH]CCC2CN1CC(C[N]C(=O)C(CC(=O)[NH3+])NC(=O))O')
m2=Chem.AddHs(m)
AllChem.EmbedMolecule(m2)
mb=Chem.MolToMolBlock(m2)
p=py3Dmol.view(width=400,height=400)
p.addModel(mb,'sdf')
p.setStyle({'stick':{},'sphere':{"scale":0.3}})
p.zoomTo()
p.show()

In [ ]:
elements = [atom.GetSymbol() for atom in m2.GetAtoms()]
coordinates = m2.GetConformer().GetPositions()
atoms = [(element, coordinate) for element, coordinate in zip(elements, coordinates)]

pyscf_mole = gto.Mole(basis="sto-3g",charge=1)
pyscf_mole.atom = atoms
pyscf_mole.build();

mf=scf.RHF(pyscf_mole)

In [ ]:
%%capture cap 
optimize(mf)

# Practice Problems 

Part 1: Optimize the enol and keto tautomers of acetone and determine the isomerization energy. Use the simple Hartree-Fock minimal basis calculations described above. Compare the results of the RDKit to results reoptimizing with PySCF. 

Part 2: Find and fix the error in the saqinavir input geometry. 

